# ML4NLP1
## Starting Point for Exercise 1, part II

This notebook is supposed to serve as a starting point and/or inspiration when starting exercise 1, part II.

One of the goals of this exercise is o make you acquainted with **skorch**. You will probably need to consult the [documentation](https://skorch.readthedocs.io/en/stable/).

# Installing skorch and loading libraries

In [ ]:
import subprocess

# Installation on Google Colab
try:
    import google.colab
    subprocess.run(['python', '-m', 'pip', 'install', 'skorch'])
except ImportError:
    pass

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetClassifier

In [ ]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)

In [ ]:
import pandas as pd
import numpy as np
import csv
import re
import string
from collections import defaultdict

## Training a classifier and making predictions

In [ ]:
# download dataset
!gdown 1QP6YuwdKFNUPpvhOaAcvv2Pcp4JMbIRs # x_train
!gdown 1QVo7PZAdiZKzifK8kwhEr_umosiDCUx6 # x_test
!gdown 1QbBeKcmG2ZyAEFB3AKGTgSWQ1YEMn2jl # y_train
!gdown 1QaZj6bI7_78ymnN8IpSk4gVvg-C9fA6X # y_test

Downloading...
From: https://drive.google.com/uc?id=1QP6YuwdKFNUPpvhOaAcvv2Pcp4JMbIRs
To: /content/x_train.txt
100% 64.1M/64.1M [00:00<00:00, 190MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QVo7PZAdiZKzifK8kwhEr_umosiDCUx6
To: /content/x_test.txt
100% 65.2M/65.2M [00:00<00:00, 154MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QbBeKcmG2ZyAEFB3AKGTgSWQ1YEMn2jl
To: /content/y_train.txt
100% 480k/480k [00:00<00:00, 107MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QaZj6bI7_78ymnN8IpSk4gVvg-C9fA6X
To: /content/y_test.txt
100% 480k/480k [00:00<00:00, 23.5MB/s]


In [ ]:
with open(f'x_train.txt') as f:
    x_train = f.read().splitlines()
with open(f'y_train.txt') as f:
    y_train = f.read().splitlines()
with open(f'x_test.txt') as f:
    x_test = f.read().splitlines()
with open(f'y_test.txt') as f:
    y_test = f.read().splitlines()

In [ ]:
import pandas as pd
# combine x_train and y_train into one dataframe
train_df = pd.DataFrame({'text': x_train, 'label': y_train})

#combine x_test and y_test into one dataframe
test_df = pd.DataFrame({'text': x_test, 'label': y_test})

In [ ]:
# T: Please use again the train/test data that includes English, German, Dutch, Danish, Swedish and Norwegian, plus 20 additional languages of your choice (the labels can be found in the file labels.csv)
# and adjust the train/test split if needed

from sklearn.model_selection import train_test_split

# Combine train and test for initial merge
combined_df = pd.concat([train_df, test_df])

# Split combined_df into 80% training and 20% testing while stratifying by label
train_df, test_df = train_test_split(combined_df, test_size=0.2, random_state=42, stratify=combined_df['label'])

selected_labels = ['eng', 'deu', 'nld', 'dan', 'swe', 'nno', 'ace', 'afr', 'als', 'amh', 'ang', 'ara', 'arg', 'arz', 'asm',
                   'ast', 'ava', 'aym', 'azb', 'aze', 'bak', 'bar', 'bcl', 'kom', 'bel', 'jpn']

train_subset = train_df[train_df['label'].isin(selected_labels)]
test_subset = test_df[test_df['label'].isin(selected_labels)]

# Define X_train, y_train, X_test, y_test
X_train = train_subset['text']
y_train = train_subset['label']
X_test = test_subset['text']
y_test = test_subset['label']

In [ ]:
# T: use your adjusted code to encode the labels here

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC

# Preprocessing
label_encoder = LabelEncoder()
le_fitted = label_encoder.fit(y_train)

y_train = le_fitted.transform(y_train)
y_test = le_fitted.transform(y_test)

In [ ]:
print(len(le_fitted.classes_))

26


In [ ]:
# T: In the following, you can find a small (almost) working example of a neural network. Unfortunately, again, the cat messed up some of the code. Please fix the code such that it is executable.

In [ ]:
# First, we extract some simple features as input for the neural network
# vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 2), max_features=100, binary=True)
vectorizer = CountVectorizer(analyzer='char', ngram_range=(1, 1), binary=False)
X = vectorizer.fit_transform(X_train.to_numpy())

In [ ]:
X = X.astype(np.float32)
# y = y_train_dev.astype(np.int64)
y = y_train.astype(np.int64)

X.shape

(20800, 3773)

In the following, we define a vanilla neural network with two hidden layers. The output layer should have as many outputs as there are classes. In addition, it should have a nonlinearity function.

In [ ]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=200,
            nonlin=F.relu,
    ):
        super(ClassifierModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin

        self.dense0 = nn.Linear(3773, num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 50)
        self.output = nn.Linear(50, 26)

    def forward(self, X, **kwargs):
      X = self.nonlin(self.dense0(X))
      X = F.relu(self.dense1(X))
      X = self.output(X)
      return X.squeeze(dim=1)

In [ ]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    criterion=nn.CrossEntropyLoss(),
    lr=0.1,
    device='cuda',  # comment this to train with CPU
)

In [ ]:
net.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.9458       0.5474        1.4507  2.6593
      2        1.2858       0.5767        1.7106  2.2737
      3        1.2798       0.5361        1.6135  3.0214
      4        0.8908       0.7755        0.7108  2.1752
      5        0.8529       0.6851        0.9602  2.2089
      6        1.0722       0.6288        1.1909  2.2045
      7        1.2545       0.6325        1.0642  2.1938
      8        0.9495       0.8202        0.6266  3.1278
      9        0.7006       0.8034        0.6292  2.2572
     10        0.5889       0.8175        0.5894  2.2957
     11        0.5795       0.8365        0.5370  2.2505
     12        0.7443       0.8495        0.5285  2.2892
     13        0.5337       0.8654        0.4452  3.1296
     14        0.5972       0.8380        0.5055  2.2192
     15        1.7057       0.6685        1.0492  2.2666
     16        1.1500       0.6

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=3773, out_features=200, bias=True)
    (dense1): Linear(in_features=200, out_features=50, bias=True)
    (output): Linear(in_features=50, out_features=26, bias=True)
  ),
)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = net.predict(X)
accuracy = accuracy_score(y, y_pred)

print(f'Accuracy: {accuracy}')


Accuracy: 0.5506730769230769


In [ ]:
from skorch.callbacks import EarlyStopping

class Module1(nn.Module):
    def __init__(
            self,
            num_units=2500,
            nonlin=F.relu,
    ):
        super(Module1, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin

        self.dense0 = nn.Linear(3773, num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 256)
        self.output = nn.Linear(256, 26)

    def forward(self, X, **kwargs):
      X = self.nonlin(self.dense0(X))
      X = F.tanh(self.dense1(X))
      X = self.output(X)
      return X.squeeze(dim=1)

# Add early stopping
early_stopping = EarlyStopping(
    monitor='valid_loss',
    patience=10,
    threshold=0.0001,  # Minimum threshold for loss improvement
    threshold_mode='rel',
    lower_is_better=True
)

net1 = NeuralNetClassifier(
    Module1,
    max_epochs=20,
    criterion=nn.CrossEntropyLoss(),
    lr=0.001,
    optimizer=torch.optim.RMSprop,
    device='cuda',
    callbacks=[early_stopping],
)

net1.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.5324       0.8115        0.5470  2.8338
      2        0.3680       0.9161        0.2823  3.5324
      3        0.2529       0.9161        0.2645  3.4568
      4        0.2169       0.9156        0.2722  6.2322
      5        0.1913       0.9209        0.2574  3.5753
      6        0.1702       0.9219        0.2671  2.7637
      7        0.1540       0.9202        0.2621  2.6958
      8        0.1386       0.9281        0.2509  2.7752
      9        0.1288       0.9325        0.2400  3.4730
     10        0.1146       0.9327        0.2434  2.8422
     11        0.1048       0.9269        0.2553  2.8619
     12        0.0896       0.9334        0.2571  2.7953
     13        0.0864       0.9356        0.2459  3.4442
     14        0.0855       0.9339        0.2506  2.9214
     15        0.0737       0.9341        0.2507  2.8102
     16        0.0692       0.9

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Module1(
    (dense0): Linear(in_features=3773, out_features=2500, bias=True)
    (dense1): Linear(in_features=2500, out_features=256, bias=True)
    (output): Linear(in_features=256, out_features=26, bias=True)
  ),
)

In [ ]:
y_pred_1 = net1.predict(X)
accuracy_1 = accuracy_score(y, y_pred_1)

print(f'Accuracy1: {accuracy_1}')

Accuracy1: 0.9711057692307692
